In [1]:
%%time
# importing libraries

import scipy.io as sio
import numpy as np  
import matplotlib.pyplot as plt
%matplotlib inline
# import sys

# environmental settings
np.set_printoptions(threshold=1000) # adjust numbers to print out

CPU times: user 1.12 s, sys: 88.8 ms, total: 1.21 s
Wall time: 2.29 s


In [2]:
%%time

ir = sio.loadmat('ir_Microstrip4inch_10Gbps')
ir_fext = sio.loadmat('ir_Microstrip4inch_fext_10Gbps')

ir = np.array(ir['ir'])
ir_fext = np.array(ir_fext['ir_fext'])

ir = ir.reshape(-1)
ir_fext = ir_fext.reshape(-1)

CPU times: user 4.25 ms, sys: 1.6 ms, total: 5.84 ms
Wall time: 7.31 ms


In [3]:
%%time

# variable definition
N = 100000        # total number of symbol
sps = 64        # number of samples in one symbol i.e. 1UI
SNR_db = 1000   # power ratio b/w signal and noise

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.05 µs


In [4]:
%%time

# channel data generation
tx_data_digital = np.zeros((2,N))
tx_data_analog = np.zeros((2,N*sps))

# Generate two independent gaussian random signal
for i in range(N):
    data = 2*np.random.randint(2, size=(2,1)) - 1 # random signal generation
    tx_data_digital[:, i] = data.flatten()
    tx_data_analog[:, (i)*sps : (i+1)*sps] = np.repeat(data,sps,axis=1)

CPU times: user 1.43 s, sys: 24.5 ms, total: 1.45 s
Wall time: 1.45 s


In [5]:
%%time

# channel response with noise

# signal thru lossy tr line i.e. insertion loss
rx_ins_loss=np.convolve(ir,tx_data_analog[0,:])

In [ ]:
%%time

# noise generation according to target SNR and dB
sig_avg_pow = np.mean(np.square(rx_ins_loss))
sig_avg_db = 10 * np.log10(sig_avg_pow)

# calculate noise magnitude in dB and power
noise_avg_db = sig_avg_db - SNR_db
noise_avg_pow = 10 ** (noise_avg_db / 10)

# generate a sample of white noise according to normal distribution
# mean = 0; variation = noise_avg_pow
noise = np.random.normal(0,np.sqrt(noise_avg_pow),len(rx_ins_loss))

In [ ]:
%%time

print(rx_ins_loss)

[-2.78691099e-06 -6.21429419e-06 -9.87556300e-06 ...  7.60576683e-08
  6.35017763e-08  3.73995697e-08]


In [ ]:
%%time

# sum up noise with original rx signal
rx_ins_loss += noise

In [ ]:
%%time

# channel response with noise and FEXT
rx_fext = np.convolve(ir_fext,tx_data_analog[1,:])

rx_signal = rx_fext + rx_ins_loss

In [ ]:
%%time

# without CDR we have to know delay of the signal 
# delay is 461 samples under 4inch 10Gbps simulation 

# Rx signal sampling
rx_sample_data = rx_ins_loss[460:460+sps*N:sps]
rx_data_digital = np.sign(rx_sample_data)

rx_sample_data_fext = rx_signal[460:460+sps*N:sps]
rx_data_digital_fext = np.sign(rx_sample_data_fext)

In [ ]:
%%time

print(rx_data_digital)

[ 1. -1. -1. ...  1.  1.  1.]


In [ ]:
%%time

# BER measurement
tx_channel1_sub = tx_data_digital[0,:]
print(f"N: {N}")
# 1. w/o FEXT
# position error: the index element is 1 if error occurred, 0 otherwise
error_pos = np.divide(abs(tx_channel1_sub - rx_data_digital),2)
error_num = sum(error_pos)

BER = error_num / N
print("BER w/o FEXT =", BER)
print(f"# error: {error_num}")

# 2. w/ FEXT
error_pos_fext = np.divide(abs(tx_channel1_sub - rx_data_digital_fext),2)
error_num_fext = sum(error_pos_fext)

BER_fext = error_num_fext / N
print("BER w/  FEXT =", BER_fext)
print("# error: {}".format(error_num_fext))

N: 100000
BER w/o FEXT = 0.0
# error: 0.0
BER w/  FEXT = 0.0094
# error: 940.0
